# NASA Mission Data Visualization

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/STLNFTART/MotorHandPro/blob/main/notebooks/02_nasa_data_visualization.ipynb)

Interactive exploration of NASA mission simulation data with 3D visualizations.

## Features

- 3D radiation exposure visualization
- Consciousness adaptation tracking
- Mission scenario comparison
- Real-time data streaming simulation
- Interactive parameter exploration

## Setup

In [ ]:
import sys
import os

IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    !pip install -q numpy scipy matplotlib plotly pandas
    
    if not os.path.exists('/content/MotorHandPro'):
        !git clone https://github.com/STLNFTART/MotorHandPro.git /content/MotorHandPro
        os.chdir('/content/MotorHandPro')
    else:
        os.chdir('/content/MotorHandPro')
    
    !pip install -q -r requirements.txt

print("✓ Setup complete!")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import glob

print("✓ Libraries imported")

## Load NASA Mission Data

Load mission simulation data for different shield configurations.

In [ ]:
# Find all NASA-compliant mission data files
mission_files = glob.glob('nasa_compliant_moderate_shield*_mission_data.csv')

if not mission_files:
    print("⚠ No NASA mission data files found.")
    print("Generating synthetic data for demonstration...\n")
    
    # Generate synthetic mission data
    np.random.seed(42)
    t = np.linspace(0, 180, 500)  # 180 days
    
    mission_data = {
        '5gcm2': pd.DataFrame({
            'day': t,
            'radiation_dose_mSv': 50 + 20 * np.sin(t/30) + np.random.normal(0, 2, len(t)),
            'consciousness_phi': 0.618 + 0.05 * np.sin(t/20) + np.random.normal(0, 0.01, len(t)),
            'shield_gcm2': 5.0
        }),
        '10gcm2': pd.DataFrame({
            'day': t,
            'radiation_dose_mSv': 35 + 15 * np.sin(t/30) + np.random.normal(0, 1.5, len(t)),
            'consciousness_phi': 0.618 + 0.04 * np.sin(t/20) + np.random.normal(0, 0.008, len(t)),
            'shield_gcm2': 10.0
        }),
        '20gcm2': pd.DataFrame({
            'day': t,
            'radiation_dose_mSv': 20 + 10 * np.sin(t/30) + np.random.normal(0, 1, len(t)),
            'consciousness_phi': 0.618 + 0.03 * np.sin(t/20) + np.random.normal(0, 0.005, len(t)),
            'shield_gcm2': 20.0
        })
    }
else:
    print(f"✓ Found {len(mission_files)} mission data files")
    mission_data = {}
    
    for file in mission_files:
        # Extract shield thickness from filename
        shield = file.split('shield')[1].split('_')[0]
        
        try:
            df = pd.read_csv(file, comment='#')
            mission_data[shield] = df
            print(f"  Loaded: {file} ({len(df)} records)")
        except Exception as e:
            print(f"  Warning: Could not load {file}: {e}")

print(f"\n✓ Loaded {len(mission_data)} mission datasets")

## 3D Radiation Exposure Visualization

Visualize radiation dose, consciousness adaptation, and time in 3D space.

In [ ]:
# Create 3D scatter plot
fig = go.Figure()

colors = {'5gcm2': 'red', '10gcm2': 'orange', '20gcm2': 'green'}

for shield, df in mission_data.items():
    # Ensure required columns exist
    if 'day' not in df.columns:
        df['day'] = np.arange(len(df))
    if 'radiation_dose_mSv' not in df.columns:
        continue
    if 'consciousness_phi' not in df.columns:
        df['consciousness_phi'] = 0.618  # Default value
    
    fig.add_trace(go.Scatter3d(
        x=df['day'],
        y=df['radiation_dose_mSv'],
        z=df['consciousness_phi'],
        mode='markers+lines',
        name=f'Shield {shield}',
        marker=dict(
            size=3,
            color=colors.get(shield, 'blue'),
            opacity=0.7
        ),
        line=dict(
            color=colors.get(shield, 'blue'),
            width=2
        ),
        hovertemplate='<b>Shield: ' + shield + '</b><br>' +
                      'Day: %{x}<br>' +
                      'Radiation: %{y:.2f} mSv<br>' +
                      'Φ: %{z:.4f}<br>' +
                      '<extra></extra>'
    ))

fig.update_layout(
    title='3D Mission Trajectory: Radiation Exposure vs Consciousness Adaptation',
    scene=dict(
        xaxis_title='Mission Day',
        yaxis_title='Radiation Dose (mSv)',
        zaxis_title='Consciousness Φ',
        camera=dict(
            eye=dict(x=1.5, y=1.5, z=1.3)
        )
    ),
    height=700,
    hovermode='closest'
)

fig.show()

## Time Series Comparison

Compare radiation exposure across different shield configurations.

In [ ]:
fig = make_subplots(
    rows=2, cols=1,
    subplot_titles=('Radiation Dose Over Time', 'Consciousness Adaptation'),
    vertical_spacing=0.12
)

for shield, df in mission_data.items():
    if 'radiation_dose_mSv' in df.columns:
        fig.add_trace(
            go.Scatter(
                x=df['day'],
                y=df['radiation_dose_mSv'],
                name=f'Shield {shield}',
                line=dict(color=colors.get(shield, 'blue'))
            ),
            row=1, col=1
        )
    
    if 'consciousness_phi' in df.columns:
        fig.add_trace(
            go.Scatter(
                x=df['day'],
                y=df['consciousness_phi'],
                name=f'Shield {shield}',
                line=dict(color=colors.get(shield, 'blue')),
                showlegend=False
            ),
            row=2, col=1
        )

fig.update_xaxes(title_text="Mission Day", row=2, col=1)
fig.update_yaxes(title_text="Dose (mSv)", row=1, col=1)
fig.update_yaxes(title_text="Φ", row=2, col=1)

fig.update_layout(
    height=700,
    title_text="Mission Timeline Analysis",
    hovermode='x unified'
)

fig.show()

## Statistical Analysis

Compute and visualize statistical metrics for each shield configuration.

In [ ]:
# Compute statistics
stats = []

for shield, df in mission_data.items():
    if 'radiation_dose_mSv' in df.columns:
        stats.append({
            'Shield': shield,
            'Mean Dose (mSv)': df['radiation_dose_mSv'].mean(),
            'Max Dose (mSv)': df['radiation_dose_mSv'].max(),
            'Std Dev': df['radiation_dose_mSv'].std(),
            'Mean Φ': df['consciousness_phi'].mean() if 'consciousness_phi' in df.columns else np.nan
        })

stats_df = pd.DataFrame(stats)

print("="*70)
print("MISSION STATISTICS")
print("="*70)
print(stats_df.to_string(index=False))
print("="*70)

# Visualize statistics
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Mean Radiation Dose', 'Maximum Dose'),
    specs=[[{'type': 'bar'}, {'type': 'bar'}]]
)

fig.add_trace(
    go.Bar(
        x=stats_df['Shield'],
        y=stats_df['Mean Dose (mSv)'],
        marker_color=['red', 'orange', 'green'],
        name='Mean'
    ),
    row=1, col=1
)

fig.add_trace(
    go.Bar(
        x=stats_df['Shield'],
        y=stats_df['Max Dose (mSv)'],
        marker_color=['red', 'orange', 'green'],
        name='Max'
    ),
    row=1, col=2
)

fig.update_yaxes(title_text="Dose (mSv)", row=1, col=1)
fig.update_yaxes(title_text="Dose (mSv)", row=1, col=2)

fig.update_layout(
    height=400,
    title_text="Shield Configuration Comparison",
    showlegend=False
)

fig.show()

## Heatmap: Radiation Intensity

Visualize radiation exposure patterns across mission duration and shield configurations.

In [ ]:
# Create heatmap data
heatmap_data = []
shields = []

for shield, df in sorted(mission_data.items()):
    if 'radiation_dose_mSv' in df.columns:
        shields.append(shield)
        # Sample data evenly across mission
        sampled = df['radiation_dose_mSv'].values[::max(1, len(df)//50)][:50]
        heatmap_data.append(sampled)

if heatmap_data:
    fig = go.Figure(data=go.Heatmap(
        z=heatmap_data,
        x=np.arange(len(heatmap_data[0])),
        y=shields,
        colorscale='Hot',
        colorbar=dict(title='Dose (mSv)'),
        hovertemplate='Shield: %{y}<br>Time Step: %{x}<br>Dose: %{z:.2f} mSv<extra></extra>'
    ))

    fig.update_layout(
        title='Radiation Exposure Heatmap',
        xaxis_title='Mission Progress',
        yaxis_title='Shield Configuration',
        height=400
    )

    fig.show()
else:
    print("No data available for heatmap")

## Live Data Streaming Simulation

Simulate real-time mission data streaming (for demonstration).

In [ ]:
import time
from IPython.display import clear_output

print("Starting live data stream simulation...\n")
print("(This will update 10 times, simulating real-time data)\n")

# Initialize streaming data
stream_t = []
stream_dose = []

for i in range(10):
    # Simulate new data point
    stream_t.append(i)
    stream_dose.append(30 + 10 * np.sin(i/2) + np.random.normal(0, 2))
    
    # Clear output and plot
    clear_output(wait=True)
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=stream_t,
        y=stream_dose,
        mode='lines+markers',
        name='Live Stream',
        line=dict(color='cyan', width=3),
        marker=dict(size=8)
    ))
    
    fig.update_layout(
        title=f'Live Mission Data Stream (Update {i+1}/10)',
        xaxis_title='Time',
        yaxis_title='Radiation Dose (mSv)',
        height=400,
        yaxis=dict(range=[0, 60])
    )
    
    fig.show()
    
    print(f"Current dose: {stream_dose[-1]:.2f} mSv")
    print(f"Mean dose:    {np.mean(stream_dose):.2f} mSv")
    print(f"Trend:        {'↑ Increasing' if len(stream_dose) > 1 and stream_dose[-1] > stream_dose[-2] else '↓ Decreasing'}")
    
    if i < 9:
        time.sleep(0.5)  # Wait before next update

print("\n✓ Stream simulation complete!")

## Export Data

Save processed data for further analysis.

In [ ]:
# Combine all mission data
combined = []

for shield, df in mission_data.items():
    df_copy = df.copy()
    df_copy['shield'] = shield
    combined.append(df_copy)

if combined:
    combined_df = pd.concat(combined, ignore_index=True)
    
    # Save to CSV
    output_file = 'nasa_mission_analysis_combined.csv'
    combined_df.to_csv(output_file, index=False)
    
    print(f"✓ Saved combined analysis to: {output_file}")
    print(f"  Total records: {len(combined_df)}")
    print(f"  Columns: {', '.join(combined_df.columns)}")
else:
    print("No data to export")

## Next Steps

- **03_parameter_analysis.ipynb**: Advanced parameter sweeps and optimization
- **04_gpu_acceleration.ipynb**: GPU-accelerated simulations (Brev only)

## Learn More

- [NASA Data Pipeline Documentation](../NASA_DATA_PIPELINE.md)
- [Visualization Guide](../VISUALIZATION_IMPLEMENTATION_GUIDE.md)
- [GitHub Repository](https://github.com/STLNFTART/MotorHandPro)

---

**Patent Pending:** U.S. Provisional Patent Application No. 63/842,846  
© 2025 Donte Lightfoot - The Phoney Express LLC / Locked In Safety